In [ ]:
!pip install videohash >> None
!pip install opencv-python >> None
!pip install line_profiler >> None
!pip install memory_profiler >> None
!pip install ffmpeg-python >> None

In [ ]:
from videohash import VideoHash
import time
import numpy as np
import ffmpeg
from skimage.metrics import structural_similarity as ssim
import cv2
from sklearn.cluster import KMeans
from PIL import Image
import imagehash
import networkx as nx
import os
import urllib.request
from moviepy.editor import VideoFileClip
import face_recognition
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

import cProfile
import profile
from memory_profiler import profile

In [ ]:
cProfile.run('my_function()')
profile.run('my_function()')
timeit.timeit('my_function()', globals=globals(), number=1000)

# line profiler
@profile
def my_function():
    # Ваш код

my_function()

# memory profiler
@profile
def my_function():
    # Ваш код

my_function()

# 0. Сроавнение по Hash

In [ ]:
%time

url1 = "https://user-images.githubusercontent.com/64683866/168872267-7c6682f8-7294-4d9a-8a68-8c6f44c06df6.mp4"
url2 = "https://user-images.githubusercontent.com/64683866/168869109-1f77c839-6912-4e24-8738-42cb15f3ab47.mp4"
#url3 = "https://user-images.githubusercontent.com/64683866/148960165-a210f2d2-6c41-4349-bd8d-a4cb673bc0af.mp4"

videohash1 = VideoHash(url=url1)
videohash2 = VideoHash(url=url2)
#videohash3 = VideoHash(url=url3)

print(videohash2.is_similar(videohash1))
#print(videohash3.is_diffrent(videohash2))

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.54 µs
True


# 1. Снижение разрешения каждого кадра для облегчения обработки

In [ ]:
def reduce_resolution(input_video_path, output_video_path, scale_factor):
    cap = cv2.VideoCapture(input_video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * scale_factor)
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * scale_factor)

    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, (width, height))

        out.write(resized_frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [ ]:
input_video = '/content/4.mp4'
output_video = '/content/output_1.mp4'

In [ ]:
scale = 0.2
reduce_resolution(input_video, output_video, scale)

# 2. Выделение ключевых кадров (с целью подрезать дублирующие и не представляющие особой днамики)

In [ ]:
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    success, frame = cap.read()

    while success:
        frames.append(frame)
        success, frame = cap.read()

    cap.release()
    return frames

In [ ]:
def is_duplicate(frame1, frame2, threshold=0.95):
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    score = ssim(gray1, gray2)
    return score > threshold

In [ ]:
def remove_duplicates(frames):
    unique_frames = []

    for i in range(len(frames)):
        if i == 0 or not is_duplicate(frames[i], frames[i-1]):
            unique_frames.append(frames[i])

    return unique_frames

In [ ]:
video_path = '/content/4.mp4'
frames = extract_frames(video_path)
unique_frames = remove_duplicates(frames)
#for i, frame in enumerate(unique_frames):
#    cv2.imwrite(f'unique_frame_{i}.jpg', frame)
print(f'Общее количество кадров: {len(frames)}')
print(f'Количество уникальных кадров: {len(unique_frames)}')

# 3. Анализ цветовой палитры видео и сравнение их для выявления схожести

In [ ]:
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    success, frame = cap.read()

    while success:
        frames.append(frame)
        success, frame = cap.read()

    cap.release()
    return frames

In [ ]:
def extract_palette(frame, n_colors=5):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pixels = frame.reshape(-1, 3)
    kmeans = KMeans(n_clusters=n_colors)
    kmeans.fit(pixels)
    return kmeans.cluster_centers_

In [ ]:
def compare_palettes(palette1, palette2):
    distance = np.linalg.norm(palette1 - palette2)
    return distance

In [ ]:
def find_similar_videos(video_paths):
    palettes = {}

    for video_path in video_paths:
        frames = extract_frames(video_path)
        if frames:
            palette = extract_palette(frames[0])
            palettes[video_path] = palette

    similar_videos = []

    for i in range(len(video_paths)):
        for j in range(i + 1, len(video_paths)):
            distance = compare_palettes(palettes[video_paths[i]], palettes[video_paths[j]])
            if distance < 500:
                similar_videos.append((video_paths[i], video_paths[j]))

    return similar_videos

In [ ]:
video_paths = ['/content/4.mp4', '/content/4_2.mp4']

In [ ]:
similar_videos = find_similar_videos(video_paths)

if similar_videos:
    print("Найденные похожие видео:")
    for video_pair in similar_videos:
        print(f"{video_pair[0]} и {video_pair[1]} похожи.")
else:
    print("Похожие видео не найдены.")

Найденные похожие видео:
/content/4.mp4 и /content/4_2.mp4 похожи.


# 4. Анализ частотной области (Использование преобразования Фурье для анализа частотных компонентов видео).

In [ ]:
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    success, frame = cap.read()

    while success:
        frames.append(frame)
        success, frame = cap.read()

    cap.release()
    return frames

In [ ]:
def compute_frequency(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    f_transform = np.fft.fft2(gray_frame)
    f_transform_shifted = np.fft.fftshift(f_transform)
    magnitude_spectrum = np.abs(f_transform_shifted)
    return magnitude_spectrum

In [ ]:
def compare_frequencies(freq1, freq2):
    diff = freq1 - freq2
    rmse = np.sqrt(np.mean(diff**2))
    return rmse

In [ ]:
def find_similar_videos(video_paths):
    frequencies = {}

    for video_path in video_paths:
        frames = extract_frames(video_path)
        if frames:
            freq = compute_frequency(frames[0])
            frequencies[video_path] = freq

    similar_videos = []

    for i in range(len(video_paths)):
        for j in range(i + 1, len(video_paths)):
            rmse = compare_frequencies(frequencies[video_paths[i]], frequencies[video_paths[j]])
            if rmse < 1000000:
                similar_videos.append((video_paths[i], video_paths[j]))

    return similar_videos

In [ ]:
video_paths = [
    '/content/4.mp4',
    '/content/4_2.mp4']

similar_videos = find_similar_videos(video_paths)

if similar_videos:
    print("Найденные похожие видео:")
    for video_pair in similar_videos:
        print(f"{video_pair[0]} и {video_pair[1]} похожи.")
else:
    print("Похожие видео не найдены.")

Найденные похожие видео:
/content/4.mp4 и /content/4_2.mp4 похожи.


# 5. Сравнение дубликатов видео по Average Hash (AHash)

In [ ]:
def average_hash(image, hash_size=8):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (hash_size, hash_size))
    avg = np.mean(image)
    return ''.join('1' if pixel > avg else '0' for pixel in image.flatten())

def process_video_ahash(video_path):
    cap = cv2.VideoCapture(video_path)
    hashes = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        ahash = average_hash(frame)
        hashes.append(ahash)

    cap.release()
    return hashes

def compare_hashes(hashes1, hashes2):
    matches = sum(h1 == h2 for h1, h2 in zip(hashes1, hashes2))
    return matches

# Пример использования
video_path1 = '/content/4_2.mp4'
video_path2 = '/content/5.mp4'

ahash_values1 = process_video_ahash(video_path1)
ahash_values2 = process_video_ahash(video_path2)

match_count = compare_hashes(ahash_values1, ahash_values2)
print(f"AHash совпадающих кадров: {match_count}")

AHash совпадающих кадров: 736


# 6. Difference Hash (DHash)

In [ ]:
def difference_hash(image, hash_size=8):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (hash_size + 1, hash_size))
    diff = image[:, 1:] > image[:, :-1]
    return ''.join('1' if x else '0' for x in diff.flatten())

def process_video_dhash(video_path):
    cap = cv2.VideoCapture(video_path)
    hashes = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        dhash = difference_hash(frame)
        hashes.append(dhash)

    cap.release()
    return hashes

video_path1 = '/content/4_2.mp4'
video_path2 = '/content/5.mp4'

dhash_values1 = process_video_dhash(video_path1)
dhash_values2 = process_video_dhash(video_path2)

match_count = compare_hashes(dhash_values1, dhash_values2)
print(f"DHash совпадающих кадров: {match_count}")

DHash совпадающих кадров: 679


# 7. Perceptual Hash (PHash)

In [ ]:
def perceptual_hash(image, hash_size=8):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (hash_size, hash_size))
    dct = cv2.dct(np.float32(image))
    dct_low_freq = dct[:hash_size // 2, :hash_size // 2]
    avg = np.mean(dct_low_freq)
    return ''.join('1' if pixel > avg else '0' for pixel in dct_low_freq.flatten())

def process_video_phash(video_path):
    cap = cv2.VideoCapture(video_path)
    hashes = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        phash = perceptual_hash(frame)
        hashes.append(phash)

    cap.release()
    return hashes

# Пример использования
video_path1 = '/content/4_2.mp4'
video_path2 = '/content/5.mp4'

phash_values1 = process_video_phash(video_path1)
phash_values2 = process_video_phash(video_path2)

match_count = compare_hashes(phash_values1, phash_values2)
print(f"PHash совпадающих кадров: {match_count}")

PHash совпадающих кадров: 870


# 8. Использование графов: Построение графов на основе характеристик видео для выявления дубликатов на основе связей.

In [ ]:
def average_hash(image, hash_size=8):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (hash_size, hash_size))
    avg = np.mean(image)
    return ''.join('1' if pixel > avg else '0' for pixel in image.flatten())

def process_video_ahash(video_path):
    cap = cv2.VideoCapture(video_path)
    hashes = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        ahash = average_hash(frame)
        hashes.append(ahash)

    cap.release()
    return hashes

def create_graph(videos):
    G = nx.Graph()

    for i, video1 in enumerate(videos):
        hashes1 = process_video_ahash(video1)
        G.add_node(video1, hashes=hashes1)

        for j, video2 in enumerate(videos):
            if i >= j:
                continue

            hashes2 = process_video_ahash(video2)
            similarity = sum(h1 == h2 for h1, h2 in zip(hashes1, hashes2))

            # Устанавливаем порог для создания ребра
            if similarity > 0:  # Можно изменить порог
                G.add_edge(video1, video2, weight=similarity)

    return G

def find_duplicates(G):
    components = list(nx.connected_components(G))
    return components

# Пример использования
video_files = ['/content/4_2.mp4', '/content/5.mp4']

graph = create_graph(video_files)
duplicate_groups = find_duplicates(graph)

for group in duplicate_groups:
    print("Группа дубликатов:", group)

Группа дубликатов: {'/content/4_2.mp4', '/content/5.mp4'}


# 9. Перцептивное хеширование

In [ ]:
def perceptual_hash(image, hash_size=8):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (hash_size, hash_size))
    dct = cv2.dct(np.float32(image))
    dct_low_freq = dct[:hash_size // 2, :hash_size // 2]
    avg = np.mean(dct_low_freq)
    return ''.join('1' if pixel > avg else '0' for pixel in dct_low_freq.flatten())

def process_video_phash(video_path):
    cap = cv2.VideoCapture(video_path)
    hashes = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        phash = perceptual_hash(frame)
        hashes.append(phash)

    cap.release()
    return hashes

def compare_hashes(hashes1, hashes2):
    matches = sum(h1 == h2 for h1, h2 in zip(hashes1, hashes2))
    return matches

video_path1 = '/content/4_2.mp4'
video_path2 = '/content/5.mp4'

phash_values1 = process_video_phash(video_path1)
phash_values2 = process_video_phash(video_path2)

match_count = compare_hashes(phash_values1, phash_values2)
print(f"PHash совпадающих кадров: {match_count}")

PHash совпадающих кадров: 870


# 10. Изучение глубины цвета и цветовых схем

In [ ]:
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    return frames

def calculate_color_histogram(frame, bins=256):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    h_hist = cv2.calcHist([hsv_frame], [0], None, [bins], [0, 180])
    s_hist = cv2.calcHist([hsv_frame], [1], None, [bins], [0, 256])
    v_hist = cv2.calcHist([hsv_frame], [2], None, [bins], [0, 256])

    h_hist = cv2.normalize(h_hist, None).flatten()
    s_hist = cv2.normalize(s_hist, None).flatten()
    v_hist = cv2.normalize(v_hist, None).flatten()

    return h_hist, s_hist, v_hist

def compare_histograms(hist1, hist2):
    # Используем метод сравнения гистограмм (например, корреляцию)
    h_similarity = cv2.compareHist(hist1[0], hist2[0], cv2.HISTCMP_CORREL)
    s_similarity = cv2.compareHist(hist1[1], hist2[1], cv2.HISTCMP_CORREL)
    v_similarity = cv2.compareHist(hist1[2], hist2[2], cv2.HISTCMP_CORREL)

    return (h_similarity + s_similarity + v_similarity) / 3

video_path1 = '/content/4_2.mp4'
video_path2 = '/content/5.mp4'

frames1 = extract_frames(video_path1)
frames2 = extract_frames(video_path2)

num_frames_to_compare = min(10, len(frames1), len(frames2))

similarity_scores = []
for i in range(num_frames_to_compare):
    hist1 = calculate_color_histogram(frames1[i])
    hist2 = calculate_color_histogram(frames2[i])
    similarity = compare_histograms(hist1, hist2)
    similarity_scores.append(similarity)

average_similarity = np.mean(similarity_scores)
print(f"Средняя схожесть между видео: {average_similarity}")

Средняя схожесть между видео: 0.9988098422954149


# 11. Определение и анализ объектов на кадрах с целью нахождения дубликатов по содержимому, а не только по визуальному представлению;

In [ ]:
def download_file(url, filename):
    if not os.path.isfile(filename):
        print(f"Скачивание {filename}...")
        urllib.request.urlretrieve(url, filename)
        print(f"{filename} скачан.")
    else:
        print(f"{filename} уже существует.")

def download_yolo_files():
    base_url = "https://github.com/AlexeyAB/darknet/blob/master/cfg/"
    cfg_file = "yolov3.cfg"
    weights_file = "yolov3.weights"

    download_file(f"{base_url}{cfg_file}?raw=true", cfg_file)
    download_file(f"https://pjreddie.com/media/files/{weights_file}", weights_file)

def load_yolo():
    download_yolo_files()
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return net, output_layers

def detect_objects(frame, net, output_layers):
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    boxes, confidences, class_ids = [], [], []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Порог уверенности
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    return boxes, confidences, class_ids

# Извлечение кадров из видео
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    return frames

# Сравнение объектов между видео
def compare_videos(video_path1, video_path2):
    net, output_layers = load_yolo()
    frames1 = extract_frames(video_path1)
    frames2 = extract_frames(video_path2)

    for frame1, frame2 in zip(frames1[:10], frames2[:10]):  # Сравниваем первые 10 кадров
        boxes1, _, _ = detect_objects(frame1, net, output_layers)
        boxes2, _, _ = detect_objects(frame2, net, output_layers)

        # Простой анализ: если количество объектов совпадает
        if len(boxes1) == len(boxes2):
            print("Кадры могут быть похожи.")
        else:
            print("Кадры разные.")

video_path1 = '/content/4_2.mp4'
video_path2 = '/content/5.mp4'
compare_videos(video_path1, video_path2)

Скачивание yolov3.cfg...
yolov3.cfg скачан.
Скачивание yolov3.weights...


HTTPError: HTTP Error 500: Internal Server Error

# 12. Разбивание видео на блоки, и получение суммеированного/среднего кадра и построение ветора по нему, тут надо протестить насколько надо зарезать видео что бы можно было адекватно сравнивать результаты;

In [ ]:
def download_file(url, filename):
    if not os.path.isfile(filename):
        print(f"Скачивание {filename}...")
        urllib.request.urlretrieve(url, filename)
        print(f"{filename} скачан.")
    else:
        print(f"{filename} уже существует.")

def download_yolo_files():
    base_url = "https://github.com/AlexeyAB/darknet/blob/master/cfg/"
    cfg_file = "yolov3.cfg"
    weights_file = "yolov3.weights"

    download_file(f"{base_url}{cfg_file}?raw=true", cfg_file)
    download_file(f"https://pjreddie.com/media/files/{weights_file}", weights_file)

def load_yolo():
    download_yolo_files()
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return net, output_layers

def detect_objects(frame, net, output_layers):
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    boxes, confidences, class_ids = [], [], []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    return boxes, confidences, class_ids

def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    return frames

def average_frame(block):
    return np.mean(block, axis=0).astype(np.uint8)

def compare_frames_l2(frame1, frame2):
    return cv2.norm(frame1, frame2, cv2.NORM_L2)

def compare_frames_ssim(frame1, frame2):
    frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    score, _ = ssim(frame1_gray, frame2_gray, full=True)
    return score

def compare_frames_orb(frame1, frame2):
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(frame1, None)
    kp2, des2 = orb.detectAndCompute(frame2, None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    return len(matches)

def compare_videos_combined(video_path1, video_path2, block_size=30):
    frames1 = extract_frames(video_path1)
    frames2 = extract_frames(video_path2)

    blocks1 = [frames1[i:i + block_size] for i in range(0, len(frames1), block_size)]
    blocks2 = [frames2[i:i + block_size] for i in range(0, len(frames2), block_size)]

    l2_scores = []
    ssim_scores = []
    orb_scores = []

    for block1, block2 in zip(blocks1, blocks2):
        avg_frame1 = average_frame(block1)
        avg_frame2 = average_frame(block2)

        l2_score = compare_frames_l2(avg_frame1, avg_frame2)
        l2_scores.append(l2_score)

        ssim_score = compare_frames_ssim(avg_frame1, avg_frame2)
        ssim_scores.append(ssim_score)

        orb_score = compare_frames_orb(avg_frame1, avg_frame2)
        orb_scores.append(orb_score)

    average_l2 = np.mean(l2_scores)
    average_ssim = np.mean(ssim_scores)
    total_orb_matches = np.sum(orb_scores)

    print(f"Средняя L2-схожесть между видео: {average_l2}")
    print(f"Среднее значение SSIM между видео: {average_ssim}")
    print(f"Общее количество совпадений (ORB): {total_orb_matches}")

video_path1 = '/content/4_2.mp4'
video_path2 = '/content/5.mp4'
compare_videos_combined(video_path1, video_path2, block_size=30)

Средняя L2-схожесть между видео: 458.88336560507014
Среднее значение SSIM между видео: 0.9976961212912031
Общее количество совпадений (ORB): 999


# 13. Повышение скорости видео с учетом схожести кадров, то есть на быстрой прокрутке соседдние кадры быстро меняются, значит видео можно порезать, то есть выкинуть каждый 2/3/4/5 кадры;

In [ ]:
def is_similar(frame1, frame2, threshold=0.9):
    hist1 = cv2.calcHist([frame1], [0, 1], None, [8, 8], [0, 256, 0, 256])
    hist2 = cv2.calcHist([frame2], [0, 1], None, [8, 8], [0, 256, 0, 256])
    cv2.normalize(hist1, hist1)
    cv2.normalize(hist2, hist2)
    similarity = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    return similarity > threshold

def compare_videos(video1_path, video2_path):
    cap1 = cv2.VideoCapture(video1_path)
    cap2 = cv2.VideoCapture(video2_path)

    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()

        if not ret1 or not ret2:
            break

        if is_similar(frame1, frame2):
            print("Кадры схожи")
        else:
            print("Кадры различаются")

    cap1.release()
    cap2.release()

def process_video(input_video, output_video, skip_frames=3):
    cap = cv2.VideoCapture(input_video)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

    ret, prev_frame = cap.read()
    frame_count = 0

    while ret:
        if frame_count % skip_frames == 0:
            out.write(prev_frame)

        ret, curr_frame = cap.read()
        if ret and is_similar(prev_frame, curr_frame):
            frame_count += 1
            prev_frame = curr_frame
            continue
        prev_frame = curr_frame
        frame_count += 1

    cap.release()
    out.release()

video1 = 'video1.mp4'
video2 = 'video2.mp4'
compare_videos(video1, video2)
process_video(video1, 'output_video.mp4')

# 14. Обрезка видео (по вертикали, по горизонтали, по произвольному углу, по центру, по углам и т.д. для снижения объема обрабатываемых данных, то есть не все пиксели надо обрабатывать);

In [ ]:
import cv2

def crop_video(input_video, output_video, crop_region):
    cap = cv2.VideoCapture(input_video)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, 30.0, (crop_region[2], crop_region[3]))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        cropped_frame = frame[crop_region[1]:crop_region[1]+crop_region[3],
                              crop_region[0]:crop_region[0]+crop_region[2]]

        out.write(cropped_frame)

    cap.release()
    out.release()

input_video = 'input.mp4'
output_video = 'output_cropped.mp4'

crop_region = (100, 50, 400, 300)
crop_video(input_video, output_video, crop_region)

# 15. Извлечения ключевых кадров из видео с использованием алгоритма KLT (Kanade-Lucas-Tomasi)

In [ ]:
import cv2
import numpy as np

def extract_keyframes(input_video, output_video):
    cap = cv2.VideoCapture(input_video)
    ret, prev_frame = cap.read()

    prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
    lk_params = dict(winSize=(15, 15), maxLevel=2)
    p0 = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, 30.0, (prev_frame.shape[1], prev_frame.shape[0]))

    keyframes = [prev_frame]

    while True:
        ret, curr_frame = cap.read()
        if not ret:
            break

        curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)

        p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, p0, None, **lk_params)

        good_new = p1[st == 1]
        good_old = p0[st == 1]

        if np.linalg.norm(good_new - good_old) > 10:
            keyframes.append(curr_frame)
            out.write(curr_frame)

        prev_gray = curr_gray.copy()
        p0 = good_new.reshape(-1, 1, 2)

    cap.release()
    out.release()

input_video = 'input.mp4'
output_video = 'output_keyframes.mp4'
extract_keyframes(input_video, output_video)

# 16. Применение методов нормализации освещения для уменьшения влияния различных условий освещения на сравнение

In [ ]:
import cv2

def normalize_lighting(image):
    yuv_image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)

    y_channel = yuv_image[:, :, 0]
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl1 = clahe.apply(y_channel)

    yuv_image[:, :, 0] = cl1
    normalized_image = cv2.cvtColor(yuv_image, cv2.COLOR_YUV2BGR)

    return normalized_image

def process_video(input_video, output_video):
    cap = cv2.VideoCapture(input_video)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        normalized_frame = normalize_lighting(frame)
        out.write(normalized_frame)

    cap.release()
    out.release()

input_video = 'input.mp4'
output_video = 'output_normalized.mp4'
process_video(input_video, output_video)

# 17. Использование временных меток для определения схожести видео на основе времени записи

In [ ]:
def get_video_timestamp(video_path):
    clip = VideoFileClip(video_path)
    return clip.reader.fps, clip.reader.duration, clip.reader.start_time

def compare_video_timestamps(video1, video2, time_threshold=5):
    fps1, duration1, start_time1 = get_video_timestamp(video1)
    fps2, duration2, start_time2 = get_video_timestamp(video2)

    print(f"Video 1: Start time: {start_time1}, Duration: {duration1}, FPS: {fps1}")
    print(f"Video 2: Start time: {start_time2}, Duration: {duration2}, FPS: {fps2}")

    if abs(start_time1 - start_time2) < time_threshold:
        print("Видео сняты в близкие моменты времени.")
    else:
        print("Видео сняты в разное время.")

video1 = 'video1.mp4'
video2 = 'video2.mp4'
compare_video_timestamps(video1, video2)

# 18. Использование технологий распознавания лиц для выявления совпадений в видео, что может помочь в поиске дубликатов.

In [ ]:
def process_video(input_video, output_video):
    cap = cv2.VideoCapture(input_video)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, 30.0,
                          (int(cap.get(3)), int(cap.get(4))))

    known_faces = []
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = frame[:, :, ::-1]

        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_faces, face_encoding)

            if True in matches:
                print("Совпадение найдено!")
            else:
                known_faces.append(face_encoding)

        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        out.write(frame)
        frame_count += 1

    cap.release()
    out.release()

input_video = 'input_video.mp4'
output_video = 'output_video.mp4'
process_video(input_video, output_video)

# 19. Перевод видео в ч/б и выделение только маски

In [ ]:
def process_video(input_video, output_video):

    cap = cv2.VideoCapture(input_video)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video, fourcc, 30.0,
                          (int(cap.get(3)), int(cap.get(4))))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        _, mask = cv2.threshold(gray_frame, 127, 255, cv2.THRESH_BINARY)

        out.write(cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR))

        cv2.imshow('Gray Frame', gray_frame)
        cv2.imshow('Mask', mask)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

input_video = 'input_video.mp4'
output_video = 'output_mask.mp4'
process_video(input_video, output_video)

# 20. Temporal Alignment: Методы для временного выравнивания видео и нахождения синхронных частей

In [ ]:
import cv2
import numpy as np
import librosa
from scipy.signal import correlate

def extract_audio_features(audio_path):
    y, sr = librosa.load(audio_path)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs, axis=1)  # Возвращаем средние значения MFCC

def find_matching_segments(features1, features2):
    correlation = correlate(features1, features2)
    return np.max(correlation), np.argmax(correlation)

def process_video(input_video, audio_path):
    audio_features = extract_audio_features(audio_path)

    cap = cv2.VideoCapture(input_video)
    frame_count = 0
    matching_frame_indices = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


        if frame_count % 30 == 0:
            matching_frame_indices.append(frame_count)

        cv2.imshow('Video Frame', frame)

        frame_count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    for index in matching_frame_indices:
        print(f'Кадр {index} может быть синхронизирован с аудио.')

if __name__ == "__main__":
    input_video = 'input_video.mp4'
    audio_path = 'audio_file.wav'
    process_video(input_video, audio_path)

In [ ]:
class VideoFeatureExtractor(nn.Module):
    def __init__(self):
        super(VideoFeatureExtractor, self).__init__()
        self.cnn = models.resnet50(pretrained=True)
        self.cnn.fc = nn.Identity()

    def forward(self, x):
        batch_size, num_frames, channels, height, width = x.shape
        features = []
        for i in range(num_frames):
            frame = x[:, i, :, :, :]
            feature = self.cnn(frame)
            features.append(feature)
        features = torch.stack(features, dim=1)
        return features

class KANLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(KANLayer, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h = torch.tanh(self.fc1(x))
        y = self.fc2(h)
        return y

class KANVideoComparisonModel(nn.Module):
    def __init__(self, feature_dim, hidden_dim, output_dim):
        super(KANVideoComparisonModel, self).__init__()
        self.feature_extractor = VideoFeatureExtractor()
        self.kan_layer = KANLayer(feature_dim, hidden_dim, output_dim)
        self.cosine_similarity = nn.CosineSimilarity(dim=1)

    def forward(self, video1, video2):

        features1 = self.feature_extractor(video1)
        features2 = self.feature_extractor(video2)

        kan_rep1 = self.kan_layer(features1.mean(dim=1))
        kan_rep2 = self.kan_layer(features2.mean(dim=1))

        similarity = self.cosine_similarity(kan_rep1, kan_rep2)
        return similarity

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = KANVideoComparisonModel(feature_dim=2048, hidden_dim=512, output_dim=128).to(device)

video1 = torch.randn(8, 16, 3, 224, 224).to(device)
video2 = torch.randn(8, 16, 3, 224, 224).to(device)

similarity = model(video1, video2)
print(f"Схожесть между видео: {similarity.mean().item() * 100:.2f}%")